In [ ]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

import jax
import jax.numpy as jnp

from optax import losses

from molnet import loss
from molnet.data import input_pipeline_online, input_pipeline_seg
from configs.tests import attention_test
from configs import root_dirs

2025-01-20 12:06:20.051896: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 12:06:20.053249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 12:06:20.080990: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 12:06:20.081504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 12:06:23.360884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
config = attention_test.get_config()
config.root_dir = root_dirs.get_root_dir("afms_rebias")

ds = input_pipeline_seg.get_datasets(config)['train']
batch = next(ds)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 18085879115725111131
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 3887586089452751882
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t

In [3]:
def cross_entropy_loss(logits, labels):
    """
    Computes the softmax cross-entropy loss for multi-class semantic segmentation in 3D.
    
    Args:
        logits: [D, H, W, num_classes] - Raw model outputs (logits).
        labels: [D, H, W] - Integer class labels for each voxel.
        
    Returns:
        Mean cross-entropy loss.
    """
    num_classes = logits.shape[-1]
    logits = logits.reshape(-1, num_classes)  # Flatten to [N, num_classes]
    labels = labels.flatten()                 # Flatten to [N]

    # Apply log-softmax for numerical stability
    log_probs = jax.nn.log_softmax(logits, axis=-1)

    # Gather log probabilities of the correct class using advanced indexing
    one_hot_labels = jax.nn.one_hot(labels, num_classes)
    loss = -jnp.sum(one_hot_labels * log_probs, axis=-1)

    return loss.mean()


In [6]:
rng = jax.random.PRNGKey(0)
y_true = batch['atom_map']
y_pred = jnp.zeros((y_true.shape + (5,)))

print(f"y_true: {y_true.shape}")
print(f"y_pred: {y_pred.shape}")

y_true: (16, 128, 128, 10)
y_pred: (16, 128, 128, 10, 5)


In [7]:
l = cross_entropy_loss(y_pred, y_true)

In [8]:
print(l)

0.23857065
